In [1]:
#Step 1: Import Libraries
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report
import re
import joblib
import string

In [2]:
#Step 2: Load Dataset
# We combine the drive_path with the file name.
fake = pd.read_csv('Fake.csv')
true = pd.read_csv('True.csv')

#  to check if it loaded correctly:
print("Fake news data shape:", fake.shape)
print("True news data shape:", true.shape)

Fake news data shape: (23481, 4)
True news data shape: (21417, 4)


In [3]:
fake.head()

,title,text,subject,date
0,Donald Trump Sends Out Embarrassing New Year’...,Donald Trump just couldn t wish all Americans ...,News,"December 31, 2017"
1,Drunk Bragging Trump Staffer Started Russian ...,House Intelligence Committee Chairman Devin Nu...,News,"December 31, 2017"
2,Sheriff David Clarke Becomes An Internet Joke...,"On Friday, it was revealed that former Milwauk...",News,"December 30, 2017"
3,Trump Is So Obsessed He Even Has Obama’s Name...,"On Christmas day, Donald Trump announced that ...",News,"December 29, 2017"
4,Pope Francis Just Called Out Donald Trump Dur...,Pope Francis used his annual Christmas Day mes...,News,"December 25, 2017"


In [4]:
true.head()

,title,text,subject,date
0,"As U.S. budget fight looms, Republicans flip t...",WASHINGTON (Reuters) - The head of a conservat...,politicsNews,"December 31, 2017"
1,U.S. military to accept transgender recruits o...,WASHINGTON (Reuters) - Transgender people will...,politicsNews,"December 29, 2017"
2,Senior U.S. Republican senator: 'Let Mr. Muell...,WASHINGTON (Reuters) - The special counsel inv...,politicsNews,"December 31, 2017"
3,FBI Russia probe helped by Australian diplomat...,WASHINGTON (Reuters) - Trump campaign adviser ...,politicsNews,"December 30, 2017"
4,Trump wants Postal Service to charge 'much mor...,SEATTLE/WASHINGTON (Reuters) - President Donal...,politicsNews,"December 29, 2017"


In [5]:
fake['class'] = 0
true['class'] = 1
data = pd.concat([fake, true], axis = 0)

In [6]:
data.sample(10) #prints dataset after concatenation

,title,text,subject,date,class
19029,China's Xi says will support Interpol raising ...,BEIJING (Reuters) - China will support Interpo...,worldnews,"September 26, 2017",1
3187,More Than HALF Of Americans Agree Trump Is Go...,Even though Donald Trump projects his own narc...,News,"January 2, 2017",0
3493,Clinton says Trump campaign likely guided Russ...,"RANCHO PALOS VERDES, Calif. (Reuters) - Hillar...",politicsNews,"May 31, 2017",1
13339,UK must produce 'credible' border plan to unlo...,DUBLIN (Reuters) - Irish Prime Minister Leo Va...,worldnews,"December 1, 2017",1
5912,Conservative Texas Lt. Governor Tweets Vile B...,Conservatives are already praising God for the...,News,"June 12, 2016",0
5541,"Exclusive: In call with Putin, Trump denounced...",WASHINGTON (Reuters) - In his first call as p...,politicsNews,"February 9, 2017",1
5655,Angry That Benghazi Panel Couldn’t Bury Hilla...,"The House Select Committee on Benghazi, which ...",News,"July 1, 2016",0
18460,TUCKER CARLSON UNLOADS On Dem Strategist Who P...,Tucker Carlson debated a New Jersey Democratic...,left-news,"Jun 20, 2017",0
21485,BRILLIANT: [VIDEO] SHERIFF CLARKE EXPLAINS HOW...,Nothing like a good dose of Sheriff Clarke to ...,left-news,"Aug 23, 2015",0
18687,PRO-HILLARY New York Daily News Writer DESTROY...,Gersh Kuntzman is a writer for the New York Da...,left-news,"May 4, 2017",0


In [7]:
data = data.drop(["title", "subject", "date"], axis = 1)

data.reset_index(inplace=True)

data.drop(['index'], axis = 1, inplace=True)

# Let's check the result
print("Data after dropping columns and resetting index:")
data.head()

Data after dropping columns and resetting index:


,text,class
0,Donald Trump just couldn t wish all Americans ...,0
1,House Intelligence Committee Chairman Devin Nu...,0
2,"On Friday, it was revealed that former Milwauk...",0
3,"On Christmas day, Donald Trump announced that ...",0
4,Pope Francis used his annual Christmas Day mes...,0


In [8]:
#Step 3: Clean the Text
def clean_text(text):
    text = text.lower()
    text = re.sub('\[.*?\]', '', text)
    text = re.sub('\\W', ' ', text) # Changed from '' to ' ' to keep spaces between words
    text = re.sub('https?://\S+|www\.\S+', '', text)
    text = re.sub('<.*?>+', '', text)
    text = re.sub('[%s]' % re.escape(string.punctuation), '', text)
    text = re.sub('\n', '', text)
    text = re.sub('\w*\d\w*', '', text)
    return text

data["text"] = data["text"].apply(clean_text)

# the result on the first few rows of text
print("Cleaned text examples:")
print(data["text"].head())

<>:4: SyntaxWarning: invalid escape sequence '\['
<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:10: SyntaxWarning: invalid escape sequence '\w'
<>:4: SyntaxWarning: invalid escape sequence '\['
<>:6: SyntaxWarning: invalid escape sequence '\S'
<>:10: SyntaxWarning: invalid escape sequence '\w'
C:\Users\asus\AppData\Local\Temp\ipykernel_5140\2931010484.py:4: SyntaxWarning: invalid escape sequence '\['
  text = re.sub('\[.*?\]', '', text)
C:\Users\asus\AppData\Local\Temp\ipykernel_5140\2931010484.py:6: SyntaxWarning: invalid escape sequence '\S'
  text = re.sub('https?://\S+|www\.\S+', '', text)
C:\Users\asus\AppData\Local\Temp\ipykernel_5140\2931010484.py:10: SyntaxWarning: invalid escape sequence '\w'
  text = re.sub('\w*\d\w*', '', text)


Cleaned text examples:
0    donald trump just couldn t wish all americans ...
1    house intelligence committee chairman devin nu...
2    on friday  it was revealed that former milwauk...
3    on christmas day  donald trump announced that ...
4    pope francis used his annual christmas day mes...
Name: text, dtype: object


In [9]:
# Step 4: Split Data
x = data["text"]
y = data["class"]

# Splitting the data into training and testing sets
xtrain, xtest, ytrain, ytest = train_test_split(x, y, test_size=0.25, random_state=42)

In [10]:
#Step 5: Convert Text to Numbers (TF-IDF Vectorizer)
# Initializing and using the TF-IDF Vectorizer
vectorizer = TfidfVectorizer()
xv_train = vectorizer.fit_transform(xtrain)
xv_test = vectorizer.transform(xtest)

# check the shape of the resulting matrices
print("Shape of training data:", xv_train.shape)
print("Shape of test data:", xv_test.shape)

Shape of training data: (33673, 94931)
Shape of test data: (11225, 94931)


In [11]:
print(xv_train)

<Compressed Sparse Row sparse matrix of dtype 'float64'
	with 6906276 stored elements and shape (33673, 94931)>
  Coords	Values
  (0, 75123)	0.07292604734967087
  (0, 70633)	0.010775078007365378
  (0, 78624)	0.1577191802015923
  (0, 45662)	0.43730050791969416
  (0, 65220)	0.03558850319839551
  (0, 59667)	0.05438811422857134
  (0, 86610)	0.051590324092646
  (0, 83700)	0.028770782416204434
  (0, 58228)	0.44070619259260985
  (0, 93080)	0.041785262786769244
  (0, 48883)	0.02069962689072253
  (0, 84825)	0.12495451366188598
  (0, 59825)	0.021770895791493224
  (0, 56845)	0.028048130898436576
  (0, 92648)	0.03907631582335541
  (0, 83726)	0.21214101057994175
  (0, 88217)	0.07229836510310744
  (0, 79752)	0.06912576336757992
  (0, 57377)	0.017976691071432693
  (0, 93779)	0.013497270822004127
  (0, 39420)	0.07377242096726422
  (0, 3107)	0.03601196531594601
  (0, 59959)	0.03853072077545446
  (0, 60531)	0.041432174143982724
  (0, 30101)	0.03677919530003215
  :	:
  (33672, 68804)	0.12136941452009097


In [12]:
#Step 6: Train the Model (Logistic Regression)
# Initializing and training the Logistic Regression model
lr = LogisticRegression()
lr.fit(xv_train, ytrain)

# Compute accuracies
train_accuracy = lr.score(xv_train, ytrain)
test_accuracy = lr.score(xv_test, ytest)

print(f"Training set accuracy: {train_accuracy:.16f}")
print(f"Testing set accuracy: {test_accuracy:.16f}")

# Making predictions on the test set
prediction = lr.predict(xv_test)

# Evaluating the model
print("\nClassification Report:")
print(classification_report(ytest, prediction))

# Save model and vectorizer
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

# (Optional) Save accuracies for Streamlit
with open("accuracy.txt", "w") as f:
    f.write(f"{train_accuracy},{test_accuracy}")


Training set accuracy: 0.9904374424613192
Testing set accuracy: 0.9858351893095768

Classification Report:
              precision    recall  f1-score   support

           0       0.99      0.99      0.99      5895
           1       0.98      0.99      0.99      5330

    accuracy                           0.99     11225
   macro avg       0.99      0.99      0.99     11225
weighted avg       0.99      0.99      0.99     11225



In [13]:
joblib.dump(vectorizer, "vectorizer.jb")
joblib.dump(lr, "lr_model.jb")

['lr_model.jb']